In [118]:

import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from sodapy import Socrata

In [119]:
# api endpoint
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
app_token = "0NT6cCdGg6VVAj2pRLjmAqDFu"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
entityid = "18881009142"
api_key = "gxc0jsptwdopcqxffpl21c8b"
api_secret_key = "2w9yp2sqyp3d7nxkidj8v65e6ow7mos3e7bnuc64rqaiccyc4c" 


In [120]:


# Set the parameters for pagination
limit = 1000  # Number of records per request
offset = 0    # Starting offset
all_data = [] # List to store all records

date_filter = "entityformdate >= '2023-01-01T00:00:00'"

while True:
    # Define the parameters with the current offset
    params = {
        '$limit': limit,
        '$offset': offset,
        '$where': date_filter
    }

    # Make the API request with HTTP Basic Authentication
    response = requests.get(api_url, params=params, auth=HTTPBasicAuth(username, password))
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        if not data:
            break  # Exit loop if no more data is returned
        all_data.extend(data)  # Add the retrieved data to the list
        offset += limit        # Increment the offset for the next request
    else:
        print(f'Error: {response.status_code}, {response.text}')
        break


KeyboardInterrupt: 

In [121]:

# Convert the list of all records to a pandas DataFrame
df = pd.DataFrame(all_data)
print(f'Total records retrieved: {len(df)}')

# memory usage of the dataframe
memory = df.memory_usage(deep=True).sum()
print(f'Total memory usage: {memory} bytes or {round(memory/1073741824, 2)} gigabytes')

Total records retrieved: 120000
Total memory usage: 217838949 bytes or 0.2 gigabytes


In [122]:
print(f'Min entityformdate: {min(df.entityformdate)}')
print(f'Max entityformdate: {max(df.entityformdate)} \n')
print(df.head())

Min entityformdate: 2023-01-01T00:00:00.000
Max entityformdate: 2023-07-25T00:00:00.000 

      entityid                   entityname    principaladdress1  \
0  20221952116               Pommeander LLC     5295 Eldridge St   
1  20221960515  CC Management Services Inc.      319 Aabc Unit N   
2  20221963658        OliveBlack Events LLC  11675 Pink Phlox Dr   
3  20221969060     Three Hive Strategy, LLC    10127 Phillips Rd   
4  20221973469             The Nest of Hope    18608 E Vassar Dr   

  principalcity principalstate principalzipcode principalcountry  \
0        Arvada             CO            80002               US   
1         Aspen             CO            81611               US   
2        Parker             CO            80134               US   
3     Lafayette             CO            80026               US   
4        Aurora             CO            80013               US   

    entitystatus jurisdictonofformation                 entitytype  ...  \
0  Good Standing 